In [4]:
%config IPCompleter.greedy=True

def anonymiseQty(slicedDfs, smallQuantile=0.01, bigQuantile=0.99):
    print()
    print('# Qty anonymisation')
    print()
    
    res = []
    for index, dfMonthTruth in enumerate(slicedDfs):
        dfCurrentMonth = dfMonthTruth.copy()
        uniqueItemIds = dfCurrentMonth['id_item'].unique()
        uniqueUserIds = dfCurrentMonth['id_user'].unique()

        print('Month n°' + str(index + 1) + ' ------------')
        for uniqueItem in uniqueItemIds:
            itemDf = dfCurrentMonth.loc[dfCurrentMonth['id_item'] == uniqueItem]

        #     Plot distribution
        #     plt.hist(itemDf['qty'], cumulative=True, weights = np.ones(len(itemDf['qty'])) / len(itemDf['qty']))
        #     plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
        #     plt.show()

            # Quantile 0.99
            quantile = itemDf['qty'].quantile(bigQuantile)
            outlierMedian = itemDf.loc[itemDf['qty'] > quantile]['qty'].median()
            dfCurrentMonth.loc[(dfCurrentMonth['id_item'] == uniqueItem) & (itemDf['qty'] > quantile), 'qty'] = outlierMedian

            # Quantile 0.01
            quantileS = itemDf['qty'].quantile(smallQuantile)
            outlierMedianS = itemDf.loc[itemDf['qty'] < quantileS]['qty'].median()
            dfCurrentMonth.loc[(dfCurrentMonth['id_item'] == uniqueItem) & (itemDf['qty'] < quantileS), 'qty'] = outlierMedianS

            # Make sure nobody is alone in qty
            qtyCount = dfCurrentMonth[dfCurrentMonth['id_item'] == uniqueItem].groupby('qty').size()
            for i, (qty, occ) in enumerate(qtyCount.items()):
                if occ == 1:
                    if i+1 == len(qtyCount):
                        dfCurrentMonth.loc[(dfCurrentMonth['id_item'] == uniqueItem) & (itemDf['qty'] == qty), 'qty'] = qtyCount.index[i - 1]
                    elif abs(qtyCount.index[i] - qtyCount.index[i - 1]) < abs(qtyCount.index[i + 1] - qtyCount.index[i]):
                        dfCurrentMonth.loc[(dfCurrentMonth['id_item'] == uniqueItem) & (itemDf['qty'] == qty), 'qty'] = qtyCount.index[i - 1]
                    else:
                        dfCurrentMonth.loc[(dfCurrentMonth['id_item'] == uniqueItem) & (itemDf['qty'] == qty), 'qty'] = qtyCount.index[i + 1]

        res.append(dfCurrentMonth)

    print('END Qty anonymisation')
    print()
    return res